combine linear model with no-lookback linear model 
work on dimensionality
do you really need the last rows x columns x 1 
check dimensiality of streeteye lstm
work on sin
make it 685x30*12, not 685*360
try one LSTM layer
try 1 or 2 GRU layers

- Use neural networks for time series prediction.
- Step 1 ... just do linear OLS to predict t+1 to see how it works
- Step 2 ... add a nonlinearity and hyperparameter tuning, see if any improvement
- Step 3 ... add an additional layer, regularization, dropout, multioutput to predict all 30 simultaneously
- Step 4 ... instead of OLS, predict quintiles
- Step 5 ... train 1926-1969, predict 1969m1, gradually move forward, select some simple hyperparameters, predict in test
- Step 6 ... directly optimize performance, determine model that would have given highest performance historically


In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import time 
import copy
from itertools import product

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Hide messy TensorFlow warnings
warnings.filterwarnings("ignore") #Hide messy Numpy warnings

from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score, r2_score
from sklearn.linear_model import LinearRegression, Lasso, lasso_path, lars_path, LassoLarsIC

import keras
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import Input
from keras.models import Model

from keras.layers.recurrent import LSTM, GRU
from keras.regularizers import l1
from keras.models import Sequential
from keras.models import load_model

import ffn
%matplotlib inline

import plotly as py
# print (py.__version__) # requires version >= 1.9.0
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import plotly.figure_factory as ff

init_notebook_mode(connected=True)


Using TensorFlow backend.


In [2]:
print("Loading data...")
data = pd.read_csv("30_Industry_Portfolios.csv")
data = data.set_index('yyyymm')

industries = list(data.columns)
# map industry names to col nums
ind_reverse_dict = dict([(industries[i], i) for i in range(len(industries))])

rfdata = pd.read_csv("F-F_Research_Data_Factors.csv")
rfdata = rfdata.set_index('yyyymm')
data['rf'] = rfdata['RF']

# subtract risk-free rate
# create a response variable led by 1 period to predict
for ind in industries:
    data[ind] = data[ind] - data['rf']
    data[ind+".lead"] = data[ind].shift(-1)

data = data.drop(columns=['rf'])    
data = data.dropna(axis=0, how='any')
    
data

Loading data...


,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm.lead,Servs.lead,BusEq.lead,Paper.lead,Trans.lead,Whlsl.lead,Rtail.lead,Meals.lead,Fin.lead,Other.lead
yyyymm,,,,,,,,,,,,,,,,,,,,,
192609,0.93,3.79,1.03,6.35,-1.22,0.50,-0.74,0.46,5.10,2.08,...,-0.43,-2.32,-1.41,-5.40,-2.96,-15.70,-2.52,-4.43,-5.48,-8.81
192610,-3.38,-3.63,0.74,-5.08,9.15,-5.00,-0.20,-0.89,-5.08,0.68,...,1.32,3.46,3.33,3.53,1.29,4.36,6.21,4.02,1.93,3.69
192611,6.04,6.98,4.24,1.35,-6.11,-0.85,1.56,5.11,4.89,2.80,...,1.71,5.93,6.96,-4.91,3.31,9.37,0.29,1.23,2.40,-2.62
192612,-0.79,-4.37,2.27,1.89,0.25,2.28,0.32,-0.17,5.09,6.15,...,1.63,1.83,-1.70,-2.85,1.19,-18.18,-2.95,1.27,-2.63,-4.21
192701,-1.09,0.32,-0.60,1.62,9.42,3.83,-1.31,4.80,-0.02,-1.75,...,3.71,8.64,4.59,5.20,4.92,3.23,3.21,6.60,2.71,0.02
192702,4.16,12.57,1.23,1.38,1.15,4.31,2.51,1.45,9.43,7.48,...,5.26,-8.10,3.99,-9.97,0.76,-20.77,-0.60,-2.72,1.06,1.88
192703,1.79,-13.86,5.21,0.72,-0.58,-0.38,0.60,0.71,5.77,-5.69,...,-2.38,3.19,2.85,4.68,0.49,-11.00,4.13,6.31,2.64,5.02
192704,2.47,2.60,3.76,-4.02,-1.21,1.19,5.70,2.49,2.70,0.29,...,3.05,18.03,4.80,5.06,6.62,-4.31,1.94,4.32,9.95,1.34
192705,5.83,11.32,11.50,-0.12,2.83,10.24,3.07,3.82,3.55,12.92,...,-2.40,-7.06,2.47,5.18,-2.23,-4.44,-0.96,1.01,-0.29,-4.08


In [3]:
# use data >= 195912 for consistency with paper
# but keep data > 2016 to be as current and have as much data as possible
data = data.loc[data.index[data.index > 195911]]

data

,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm.lead,Servs.lead,BusEq.lead,Paper.lead,Trans.lead,Whlsl.lead,Rtail.lead,Meals.lead,Fin.lead,Other.lead
yyyymm,,,,,,,,,,,,,,,,,,,,,
195912,2.01,0.35,-3.02,1.64,7.29,0.67,1.87,-1.97,3.08,0.74,...,0.62,-6.18,-7.93,-9.41,-4.31,-5.33,-6.09,-10.08,-4.68,-3.98
196001,-4.49,-5.71,-2.05,1.21,-5.47,-7.84,-8.53,-6.68,-10.03,-4.77,...,8.07,9.13,5.09,3.00,-0.94,1.42,4.00,1.81,-0.98,6.32
196002,3.35,-2.14,2.27,4.23,2.39,9.31,1.44,-0.02,-0.74,0.32,...,-0.21,-0.31,3.34,-2.43,-4.99,-1.37,-0.13,-3.88,0.05,-2.43
196003,-1.67,-2.94,-0.18,-0.65,2.18,-0.56,-2.59,1.26,-2.75,-6.79,...,-1.24,7.14,1.77,0.41,-2.13,0.45,-0.53,8.86,-0.64,0.55
196004,1.17,-2.16,1.35,6.46,-1.17,-1.27,0.21,1.49,-5.53,-1.10,...,3.05,-1.75,11.90,2.85,0.90,1.65,3.11,0.80,-0.45,1.02
196005,8.20,-0.52,2.44,7.28,11.67,7.74,1.74,13.50,3.40,2.10,...,-0.58,-8.07,2.39,3.50,2.17,5.96,3.41,1.03,3.72,6.41
196006,5.39,0.47,4.73,2.24,0.02,6.38,-1.59,-0.40,0.45,4.04,...,-0.03,2.84,-2.02,-4.10,-3.11,-6.16,-2.99,-1.25,0.09,-5.95
196007,-2.11,-0.79,4.60,-4.72,0.23,-0.60,-1.10,-3.99,-6.80,-3.14,...,6.94,5.69,2.71,1.18,1.98,4.51,2.85,2.05,3.47,3.48
196008,4.57,3.24,5.20,7.16,3.63,5.09,3.34,2.29,1.17,-0.84,...,-6.07,-3.53,-7.61,-7.37,-7.07,-8.44,-8.57,-1.90,-5.78,-4.21


In [4]:
# roll up X so each row contains 12 previous months * 30 cols
# create training, CV, test sets
# multi-output: simultaneously train all estimators, forcing 1st layer look for common patterns and inputs
# add regularization
# add dropout

numcols = 30
lookback = 12

X_raw = data.values[:,:numcols]
Y = data.values[12:, -numcols:]
numrows = Y.shape[0]

# each input will have shape 12 * 30
X = np.zeros([numrows, lookback, numcols])
for row in range(numrows):
    prev12 = []
    for i in range(lookback):
        prev12.append(X_raw[row + i])
    X[row] = np.vstack(prev12)

print(X.shape)
print(Y.shape)

(685, 12, 30)
(685, 30)


In [5]:
# build LSTM model
# input (360)
# -> LSTM
# -> Dropout
# -> LSTM
# -> Dropout
# -> 30 Dense linear outputs (1 per industry)

INPUT_DIM = numcols
OUTPUT_DIM = numcols

def build_model(hidden_layers = [[16, 0.0001, 0.2], 
                                 [16, 0.0001, 0.2], 
                                 [1, 0.0, 0.0]]):
    """Take list of [layer_size, reg_penalty, dropout], last layer is linear, rest LSTM"""

    main_input = Input(shape=(None, 
                              INPUT_DIM), 
                       dtype='float32', 
                       name='main_input')
    lastlayer=main_input

    n_lstms = len(hidden_layers)-1
    for i in range(n_lstms):
        layer_size, reg_penalty, dropout = hidden_layers[i]
        print("layer %d size %d, reg_penalty %.8f, dropout %.3f" % (i, layer_size, reg_penalty, dropout))

        # first n-1 layers are LSTM, return_sequences=True
        return_sequences = True
        if i == n_lstms-1:
            return_sequences= False
        LSTM_layer = GRU(layer_size, 
                         return_sequences=return_sequences,
                         kernel_regularizer=keras.regularizers.l1(reg_penalty),
                         dropout=dropout, 
                         recurrent_dropout=dropout,
                         name = "GRU%02d" % i
                        )(lastlayer)
        lastlayer = LSTM_layer
        
    layer_size, reg_penalty, dropout = hidden_layers[-1]
    if dropout:
        lastlayer = Dropout(dropout)(lastlayer)
    
    outputs = []
    for i in range(OUTPUT_DIM):
        # OUTPUT_DIM outputs
        output01 = Dense(1, 
                         activation='linear', 
                         kernel_regularizer=keras.regularizers.l1(reg_penalty),
                         name='output%02d' % i)(lastlayer)
        outputs.append(output01)
    
    model = Model(inputs=[main_input], outputs=outputs)
    print(model.summary())
    model.compile(loss="mae", metrics=['mae'], optimizer="rmsprop", loss_weights=[1.]*OUTPUT_DIM)
    return model


In [6]:
MODELPREFIX = "NN"
EPOCHS = 50
#VAL_SPLIT = 0.2
BATCH_SIZE = 32
LOOKBACK = 128
BATCH_SIZE = 64
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

def run_experiment (layer1_size = 32, 
                    layer2_size = 32, 
                    layer1_reg_penalty=0.0001,
                    layer2_reg_penalty=0.0001,
                    layer1_dropout=0.25,
                    layer2_dropout=0.25,
                    layer3_reg_penalty=0.0001,
                    layer3_dropout=0.25
                   ):

    start = time.time()

    # generate k-folds
    n_splits = 5
    kf = KFold(n_splits=n_splits)
    kf.get_n_splits(X)
    last_indexes = []
    for train_index, test_index in kf.split(X):
        # use test_index as last index to train
        last_index = test_index[-1] + 1
        last_indexes.append(last_index)

    print("%s Generate splits %s" % (time.strftime("%H:%M:%S"), str([i for i in last_indexes])))
    
    avg_bests = []

    for i in range(1, n_splits-1):
        
        print("%s Build model" % (time.strftime("%H:%M:%S")))
        
        model = build_model([[layer1_size, layer1_reg_penalty, layer1_dropout],
                             [layer2_size, layer2_reg_penalty, layer2_dropout],
                             [1, layer3_reg_penalty, layer3_dropout],
                            ])
        print("Compile time : %s" % str(time.time() - start))
        print("Starting to train : %s" % (time.strftime("%H:%M:%S")))

        models = []
        losses = []
        scores = []
        count = 0        
        # skip kfold 0 so you start with train 2x size of eval set
        last_train_index = last_indexes[i]
        last_xval_index = last_indexes[i+1]

        # set up train, xval
        # train from beginning to last_train_index
        print("Training indexes 0 to %d" % (last_train_index-1))
        X_fit = X[:last_train_index]
        Y_fit = Y[:last_train_index]
        # xval from last_train_index to last_xval_index
        print("Cross-validating indexes %d to %d" % (last_train_index, last_xval_index -1 ))
        X_xval = X[last_train_index:last_xval_index]
        Y_xval = Y[last_train_index:last_xval_index]

        responses = []
        for i in range(numcols):
            responses.append(Y_fit[:,i])
        # train for EPOCHS
        for epoch in range(EPOCHS):
            fit = model.fit(
                X_fit,
                responses,
                batch_size=BATCH_SIZE,
                #validation_split=VAL_SPLIT,
                epochs=1,
                verbose=0)
            
            train_loss = fit.history['loss'][-1]
            # evaluate ... run prediction, calc MSE by industry, and average
            y_xval_pred = np.array(model.predict(X_xval))
            y_xval_pred = y_xval_pred.reshape(Y_xval.T.shape)
            y_xval_pred = y_xval_pred.T
            mse_list = []
            for i in range(len(industries)):
                mse_list.append(mean_squared_error(Y_xval[:,i], y_xval_pred[:,i]))
            xval_score = np.mean(np.array(mse_list))            
            
            losses.append(train_loss)
            scores.append(xval_score)
            models.append(copy.copy(model))

            bestloss_index = np.argmin(scores)
            bestloss_value = scores[bestloss_index]

            sys.stdout.write('.')
            count += 1
            if count % 80 == 0:
                print("")
                print("%s Still training" % (time.strftime("%H:%M:%S")))
            sys.stdout.flush()            
            
            # stop if loss rises by 20% from best
            if xval_score / bestloss_value > 1.2:
                print("Stopping early..." )
                break

        # choose model with lowest xval loss
        print("")
        print ("%s Best Xval loss epoch %d, value %f" % (time.strftime("%H:%M:%S"), bestloss_index, bestloss_value))
        avg_bests.append(bestloss_value)
        model = models[bestloss_index]
    
    print ("Last Xval loss %f" % (bestloss_value))
    avg_loss = np.mean(np.array(avg_bests))
    print ("Avg Xval loss %f" % avg_loss)
    print("--------------------------------------------------------------------------------")
    return avg_loss


In [7]:
run_experiment()

18:32:05 Generate splits [137, 274, 411, 548, 685]
18:32:05 Build model
layer 0 size 32, reg_penalty 0.00010000, dropout 0.250
layer 1 size 32, reg_penalty 0.00010000, dropout 0.250
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, None, 30)     0                                            
__________________________________________________________________________________________________
GRU00 (GRU)                     (None, None, 32)     6048        main_input[0][0]                 
__________________________________________________________________________________________________
GRU01 (GRU)                     (None, 32)           6240        GRU00[0][0]                      
__________________________________________________________________________________________________
dropout_1 (Dropout)       

Compile time : 35.7515180111
Starting to train : 18:32:41
Training indexes 0 to 410
Cross-validating indexes 411 to 547
..................................................
18:33:25 Best Xval loss epoch 18, value 40.952916
18:33:25 Build model
layer 0 size 32, reg_penalty 0.00010000, dropout 0.250
layer 1 size 32, reg_penalty 0.00010000, dropout 0.250
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, None, 30)     0                                            
__________________________________________________________________________________________________
GRU00 (GRU)                     (None, None, 32)     6048        main_input[0][0]                 
__________________________________________________________________________________________________
GRU01 (GRU)                     (None, 32)           6

38.34921413101546

In [23]:
# run in big xval loop
# do predictions
# compute mse
# pick best hyperparameters

layer1_sizes = [8, 16, 32]
layer2_sizes = [4, 8, 16]
layer1_reg_penalties = [0.0]
layer2_reg_penalties = [0.0]
layer3_reg_penalties = [0.001, 0.0001]
layer1_dropouts = [0.0, 0.25]
layer2_dropouts = [0.0, 0.25]
layer3_dropouts = [0.0, 0.25]

hyperparameter_combos = list(product(layer1_sizes,
                                     layer2_sizes,
                                     layer1_reg_penalties,
                                     layer2_reg_penalties,
                                     layer3_reg_penalties,
                                     layer1_dropouts,
                                     layer2_dropouts,
                                     layer3_dropouts,
                                    ))

print("%s Running %d experiments" % (time.strftime("%H:%M:%S"), len(hyperparameter_combos)))

experiments = {}

for counter, param_list in enumerate(hyperparameter_combos):
    layer1_size, layer2_size, layer1_reg_penalty, layer2_reg_penalty, layer3_reg_penalty, layer1_dropout, layer2_dropout, layer3_dropout = param_list
    print("%s Running experiment %d of %d" % (time.strftime("%H:%M:%S"), counter+1, len(hyperparameter_combos)))
    key = (layer1_size, layer1_reg_penalty, layer1_dropout, layer2_size, layer2_reg_penalty, layer2_dropout, layer3_reg_penalty, layer3_dropout)
    experiments[key] = run_experiment(layer1_size=layer1_size,
                                      layer1_reg_penalty=layer1_reg_penalty,
                                      layer1_dropout=layer1_dropout,
                                      layer2_size=layer2_size,
                                      layer2_reg_penalty=layer2_reg_penalty,
                                      layer2_dropout=layer2_dropout,
                                      layer3_reg_penalty=layer3_reg_penalty,
                                      layer3_dropout=layer3_dropout
                                     )
                            

23:44:54 Running 144 experiments
23:44:54 Running experiment 1 of 144
23:44:54 Generate splits [137, 274, 411, 548, 685]
23:44:54 Build model
layer 0 size 16, reg_penalty 0.00000000, dropout 0.000
layer 1 size 8, reg_penalty 0.00000000, dropout 0.000
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, None, 30)     0                                            
__________________________________________________________________________________________________
GRU00 (GRU)                     (None, None, 16)     2256        main_input[0][0]                 
__________________________________________________________________________________________________
GRU01 (GRU)                     (None, 8)            600         GRU00[0][0]                      
________________________________________________________

Compile time : 34.274173975
Starting to train : 23:45:28
Training indexes 0 to 410
Cross-validating indexes 411 to 547
..................................................
23:46:12 Best Xval loss epoch 37, value 40.753391
23:46:12 Build model
layer 0 size 16, reg_penalty 0.00000000, dropout 0.000
layer 1 size 8, reg_penalty 0.00000000, dropout 0.000
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, None, 30)     0                                            
__________________________________________________________________________________________________
GRU00 (GRU)                     (None, None, 16)     2256        main_input[0][0]                 
__________________________________________________________________________________________________
GRU01 (GRU)                     (None, 8)            600

Compile time : 0.520694971085
Starting to train : 23:47:08
Training indexes 0 to 273
Cross-validating indexes 274 to 410
..................................................
23:47:44 Best Xval loss epoch 38, value 33.114693
23:47:44 Build model
layer 0 size 16, reg_penalty 0.00000000, dropout 0.000
layer 1 size 8, reg_penalty 0.00000000, dropout 0.250
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, None, 30)     0                                            
__________________________________________________________________________________________________
GRU00 (GRU)                     (None, None, 16)     2256        main_input[0][0]                 
__________________________________________________________________________________________________
GRU01 (GRU)                     (None, 8)            6

Compile time : 83.8765919209
Starting to train : 23:48:31
Training indexes 0 to 547
Cross-validating indexes 548 to 684
..................................................
23:49:29 Best Xval loss epoch 18, value 42.216665
Last Xval loss 42.216665
Avg Xval loss 38.868588
--------------------------------------------------------------------------------
23:49:29 Running experiment 3 of 144
23:49:29 Generate splits [137, 274, 411, 548, 685]
23:49:29 Build model
layer 0 size 16, reg_penalty 0.00000000, dropout 0.250
layer 1 size 8, reg_penalty 0.00000000, dropout 0.000
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, None, 30)     0                                            
__________________________________________________________________________________________________
GRU00 (GRU)                     (No

Compile time : 38.2785968781
Starting to train : 23:50:07
Training indexes 0 to 410
Cross-validating indexes 411 to 547
..................................................
23:50:55 Best Xval loss epoch 22, value 41.250937
23:50:55 Build model
layer 0 size 16, reg_penalty 0.00000000, dropout 0.250
layer 1 size 8, reg_penalty 0.00000000, dropout 0.000
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, None, 30)     0                                            
__________________________________________________________________________________________________
GRU00 (GRU)                     (None, None, 16)     2256        main_input[0][0]                 
__________________________________________________________________________________________________
GRU01 (GRU)                     (None, 8)            60

...

Compile time : 230.990607977
Starting to train : 14:17:14
Training indexes 0 to 410
Cross-validating indexes 411 to 547
..................................................
14:21:18 Best Xval loss epoch 6, value 41.349576
14:21:18 Build model
layer 0 size 64, reg_penalty 0.00010000, dropout 0.000
layer 1 size 16, reg_penalty 0.00010000, dropout 0.000
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, None, 30)     0                                            
__________________________________________________________________________________________________
GRU00 (GRU)                     (None, None, 64)     18240       main_input[0][0]                 
__________________________________________________________________________________________________
GRU01 (GRU)                     (None, 16)           38

Compile time : 0.539971828461
Starting to train : 14:25:39
Training indexes 0 to 273
Cross-validating indexes 274 to 410
..................................................
14:29:32 Best Xval loss epoch 36, value 32.878082
14:29:32 Build model
layer 0 size 64, reg_penalty 0.00010000, dropout 0.000
layer 1 size 16, reg_penalty 0.00010000, dropout 0.250
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, None, 30)     0                                            
__________________________________________________________________________________________________
GRU00 (GRU)                     (None, None, 64)     18240       main_input[0][0]                 
__________________________________________________________________________________________________
GRU01 (GRU)                     (None, 16)           

Compile time : 484.663810015
Starting to train : 14:33:43
Training indexes 0 to 547
Cross-validating indexes 548 to 684
..................................................
14:38:06 Best Xval loss epoch 9, value 42.097824
Last Xval loss 42.097824
Avg Xval loss 38.711004
--------------------------------------------------------------------------------
14:38:06 Running experiment 127 of 144
14:38:06 Generate splits [137, 274, 411, 548, 685]
14:38:06 Build model
layer 0 size 64, reg_penalty 0.00010000, dropout 0.250
layer 1 size 16, reg_penalty 0.00010000, dropout 0.000
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, None, 30)     0                                            
__________________________________________________________________________________________________
GRU00 (GRU)                     (

Compile time : 236.536522865
Starting to train : 14:42:03
Training indexes 0 to 410
Cross-validating indexes 411 to 547
..........................

ResourceExhaustedError: OOM when allocating tensor with shape[12,64,64]
	 [[Node: training_382/RMSprop/gradients/GRU01_385/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3_grad/TensorArrayGatherV3 = TensorArrayGatherV3[_class=["loc:@GRU01_385/transpose"], dtype=DT_FLOAT, element_shape=<unknown>, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_382/RMSprop/gradients/GRU01_385/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3_grad/TensorArrayGrad/TensorArrayGradV3, GRU01_385/TensorArrayUnstack/range, training_382/RMSprop/gradients/GRU01_385/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3_grad/TensorArrayGrad/gradient_flow)]]

Caused by op u'training_382/RMSprop/gradients/GRU01_385/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3_grad/TensorArrayGatherV3', defined at:
  File "/home/ubuntu/anaconda2/envs/tensorflow/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/ubuntu/a

In [25]:
experiments

{(16, 0, 0, 8, 0): 42.21666518487849,
 (16, 0, 0, 8, 0.0001): 42.17057733103664,
 (16, 0, 0, 16, 0): 42.04257502122644,
 (16, 0, 0, 16, 0.0001): 41.66337853263284,
 (16, 0, 0, 32, 0): 41.9255297005299,
 (16, 0, 0, 32, 0.0001): 41.949294101465505,
 (16, 0, 0.25, 8, 0): 41.985847534943225,
 (16, 0, 0.25, 8, 0.0001): 42.00113899365784,
 (16, 0, 0.25, 16, 0): 41.809698765466116,
 (16, 0, 0.25, 16, 0.0001): 41.856498351736406,
 (16, 0, 0.25, 32, 0): 41.737382024056615,
 (16, 0, 0.25, 32, 0.0001): 41.551725092128365,
 (16, 0.0001, 0, 8, 0): 41.85243171420351,
 (16, 0.0001, 0, 8, 0.0001): 42.10118607132175,
 (16, 0.0001, 0, 16, 0): 42.01366644888894,
 (16, 0.0001, 0, 16, 0.0001): 42.11677858845035,
 (16, 0.0001, 0, 32, 0): 41.30560054981353,
 (16, 0.0001, 0, 32, 0.0001): 42.166361085563445,
 (16, 0.0001, 0.25, 8, 0): 41.580846809504216,
 (16, 0.0001, 0.25, 8, 0.0001): 41.96762025517861,
 (16, 0.0001, 0.25, 16, 0): 41.580755631446515,
 (16, 0.0001, 0.25, 16, 0.0001): 41.558779059644706,
 (16, 

In [27]:
# make a dataframe with columns as loss keys + loss value
flatlist = [list(l[0]) + [l[1]] for l in experiments.items()]
lossframe = pd.DataFrame(flatlist, columns=["layer1_size",
                                            "layer1_reg_penalty",
                                            "layer1_dropout",
                                            "layer2_size",
                                            "layer2_reg_penalty",
#                                            "layer2_dropout",
#                                            "layer3_reg_penalty",
#                                            "layer3_dropout",
                                            "loss"
                                           ])
lossframe = lossframe.sort_values(['loss'])
lossframe


,layer1_size,layer1_reg_penalty,layer1_dropout,layer2_size,layer2_reg_penalty,loss
34,32,0.0001,0.00,8,0.0001,41.180754
19,32,0.0000,0.25,32,0.0000,41.293897
54,16,0.0001,0.00,32,0.0000,41.305601
14,16,0.0001,0.25,32,0.0000,41.331479
26,32,0.0000,0.00,8,0.0001,41.356272
8,32,0.0000,0.25,16,0.0001,41.465129
38,32,0.0000,0.25,8,0.0001,41.511938
13,16,0.0000,0.25,32,0.0001,41.551725
1,16,0.0001,0.25,16,0.0001,41.558779
15,16,0.0001,0.25,16,0.0000,41.580756


In [64]:

#sort each column by mean loss
pd.DataFrame(lossframe.groupby(['layer2_size'])['loss'].mean())

,loss
layer2_size,
8,41.951932
16,41.897517
32,41.765835


In [34]:
pd.DataFrame(lossframe.groupby(['layer1_reg_penalty'])['loss'].mean())

,loss
layer1_reg_penalty,
0.0000,41.889373
0.0001,41.880086


In [67]:
pd.DataFrame(lossframe.groupby(['layer1_dropout'])['loss'].mean())

,loss
layer1_dropout,
0.00,41.959387
0.25,41.807814


In [36]:
lossframe.pivot_table(index=['layer1_size'], columns=['layer2_size'], values=['loss'])


loss                      
layer2_size         8          16         32
layer1_size                                 
16           41.984539  41.830266  41.755913
32           41.838402  41.856329  41.775756
64           42.032856  42.021446        NaN

In [63]:
# pivot and run a heatmap

def plot_matrix(lossframe, x_labels, y_labels, x_suffix="", y_suffix=""):

    pivot = lossframe.pivot_table(index=[x_labels], columns=[y_labels], values=['loss'])
    xaxis = ["%d %s" % (i, x_suffix) for i in pivot.columns.levels[1].values]
    yaxis = ["%d %s" % (i, y_suffix) for i in pivot.index.values]
    print(xaxis, yaxis)
    """plot a heat map of a matrix"""
    chart_width=640
    chart_height=480
    
    layout = Layout(
        title="%s v. %s" % (x_labels, y_labels),
        height=chart_height,
        width=chart_width,     
        margin=dict(
            l=150,
            r=30,
            b=120,
            t=100,
        ),
        xaxis=dict(
            title=y_labels,
            tickfont=dict(
                family='Arial, sans-serif',
                size=10,
                color='black'
            ),
        ),
        yaxis=dict(
            title=x_labels,
            tickfont=dict(
                family='Arial, sans-serif',
                size=10,
                color='black'
            ),
        ),
    )
    
    data = [Heatmap(z=pivot.values,
                    x=xaxis,
                    y=yaxis,
                    colorscale=[[0, 'rgb(0,0,255)', [1, 'rgb(255,0,0)']]],
                   )
           ]

    fig = Figure(data=data, layout=layout)
    return iplot(fig, link_text="")

plot_matrix(lossframe, "layer1_size", "layer2_size", x_suffix=" units", y_suffix=" units")


(['8  units', '16  units', '32  units'], ['16  units', '32  units', '64  units'])


In [77]:
model.predict(X[-1].reshape(1,12,30))

[array([[-0.24277441]], dtype=float32),
 array([[0.06869789]], dtype=float32),
 array([[0.1825534]], dtype=float32),
 array([[-0.02720911]], dtype=float32),
 array([[0.39224246]], dtype=float32),
 array([[0.16400138]], dtype=float32),
 array([[-0.34573263]], dtype=float32),
 array([[0.22301225]], dtype=float32),
 array([[0.01512091]], dtype=float32),
 array([[0.8061534]], dtype=float32),
 array([[0.00223062]], dtype=float32),
 array([[-0.42167595]], dtype=float32),
 array([[-0.01371424]], dtype=float32),
 array([[0.04307675]], dtype=float32),
 array([[-0.22560821]], dtype=float32),
 array([[0.0204489]], dtype=float32),
 array([[0.3449584]], dtype=float32),
 array([[0.48786777]], dtype=float32),
 array([[0.86726505]], dtype=float32),
 array([[0.2735452]], dtype=float32),
 array([[0.00913496]], dtype=float32),
 array([[0.14963274]], dtype=float32),
 array([[0.50360084]], dtype=float32),
 array([[-0.30614635]], dtype=float32),
 array([[0.00804199]], dtype=float32),
 array([[0.07174452]], 

In [78]:
X.shape

(685, 12, 30)

In [8]:
# train model for EPOCHS on X[:-1]
# predict Y[-1] using X[-1], return prediction

BATCH_SIZE=32
EPOCHS=15
OUTPUT_DIM=30

def fit_predict(model, X, Y, epochs=EPOCHS):
    """fit Ys v. X using n-1 rows
    predict Ys on X using nth row
    return predictions for last month"""
    
    # keep last row to predict against
    X_predict = X[-1]
    X_predict = X_predict.reshape(1,X.shape[1],X.shape[2])
    # fit on remaining rows
    X_fit = X[:-1]
    Y_fit = Y[:-1]

    model.compile(loss="mae", metrics=['mae'], optimizer="rmsprop", loss_weights=[1.]*OUTPUT_DIM)
    #model.compile(loss="mse", metrics=['mse'], optimizer="rmsprop")

    Ys = []
    for i in range(OUTPUT_DIM):
        Ys.append(Y_fit[:,i])
        
    fit = model.fit(
        X_fit,
        Ys,
        batch_size=BATCH_SIZE,
        epochs=epochs,
        verbose=0)
    
    Y_predict = model.predict(X_predict)
        
    return Y_predict

In [9]:
# initialize predictions matrix
STARTMONTH=121
P = np.zeros(Y.shape)

model = build_model([[32, 0.0001, 0.25],
                     [32, 0.0001, 0.25],
                     [1, 0.0001, 0.25],
                    ])
print("%s Start backtest training" % (time.strftime("%H:%M:%S")))

count = 0
for month_index in range(STARTMONTH, X.shape[0]+1):
    predictions = fit_predict(model, X[:month_index, :], Y[:month_index])
    predictions = [pred[0,0] for pred in predictions]
    
    try:
        P[month_index]= predictions
        sys.stdout.write('.')
        count += 1
        if count % 80 == 0:
            print("")
            print("%s Still training" % (time.strftime("%H:%M:%S")))
        sys.stdout.flush()
    except IndexError:
        # I want to run the fit and see the expected R-squared
        # but we are storing the predictions in row of the month predicted
        # so we have no row to store the last prediction (2017-01)
        print("\nlast prediction not stored")

print("%s End backtest training" % (time.strftime("%H:%M:%S")))


layer 0 size 32, reg_penalty 0.00010000, dropout 0.250
layer 1 size 32, reg_penalty 0.00010000, dropout 0.250
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, None, 30)     0                                            
__________________________________________________________________________________________________
GRU00 (GRU)                     (None, None, 32)     6048        main_input[0][0]                 
__________________________________________________________________________________________________
GRU01 (GRU)                     (None, 32)           6240        GRU00[0][0]                      
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 32)           0           GRU01[0][0]                      

KeyboardInterrupt: 

In [15]:
R = np.zeros(P.shape[0])
numstocks = 6 # top quintile (and bottom)

for month_index in range(STARTMONTH, X.shape[0]):
        # get indexes of sorted smallest to largest
        select_array = np.argsort(P[month_index])
        # leftmost 6
        short_indexes = select_array[:numstocks]
        # rightmost 6
        long_indexes = select_array[-numstocks:]
        # compute equal weighted long/short return
        R[month_index] = np.mean(X[month_index, long_indexes])/2 - np.mean(X[month_index, short_indexes])/2

results = R[STARTMONTH:]
index = pd.date_range('01/01/1970',periods=564, freq='M')
perfdata = pd.DataFrame(results,index=index,columns=['Returns'])
perfdata['Equity'] = 100 * np.cumprod(1 + results / 100)

stats = perfdata['Equity'].calc_stats()

pd.DataFrame([stats.stats.loc['start'],
              stats.stats.loc['end'],
              stats.stats.loc['cagr'],
              stats.stats.loc['yearly_vol'],
              stats.stats.loc['yearly_sharpe'],
              stats.stats.loc['max_drawdown'],
              ffn.core.calc_sortino_ratio(perfdata.Returns, rf=0, nperiods=564, annualize=False),
             ],
            index = ['start',
                     'end',
                     'cagr',
                     'yearly_vol',
                     'yearly_sharpe',
                     'max_drawdown',
                     'sortino',
                    ],
            columns=['Value'])


,Value
start,1970-01-31 00:00:00
end,2016-12-31 00:00:00
cagr,0.0219107
yearly_vol,0.0544492
yearly_sharpe,0.460855
max_drawdown,-0.259784
sortino,0.162216


In [ ]:
perf = 100 * np.cumprod(1 + results / 100)

x_coords = np.linspace(1970, 2016, perf.shape[0])

trace1 = Scatter(
    x = x_coords,
    y = perf,
    name = 'Growth of $1',    
)

layout = Layout(
    yaxis=dict(
        type='log',
        autorange=True
    )
)
data = [trace1]

fig = Figure(data=data, layout=layout)

iplot(fig)

In [ ]:
curr_frame = X[0]
predictions = [z[0] for z in curr_frame]

for i in range(X.shape[0]):
    predictions.append(model.predict(curr_frame[newaxis,:,:])[0,0])
    curr_frame = np.array(predictions[-window_size:]).reshape([window_size,1])    

# truncate the ones which aren't predictions
predictions = np.array(predictions[window_size:])


In [ ]:
x_coords = np.linspace(0, 1, y.shape[0])

trace1 = Scatter(
    x = x_coords,
    y = y,
    name = 'Training data',    
)
trace2 = Scatter(    
    x = x_coords,
    y = predictions,
    name = 'Predictions',

)

data = [trace1, trace2]

iplot(data, filename='basic-line')